## Imports

### Run the following cells before navigating through the notebook

In [1]:
%env SM_FRAMEWORK=tf.keras

env: SM_FRAMEWORK=tf.keras


In [ ]:
#Optional Code to reset GPU
from numba import cuda 
device = cuda.get_current_device()
print(device)
#device.reset()

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import warnings

def function_that_warns():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    function_that_warns()  # this will not show a warning

In [3]:
import os
import numpy as np
import sys
import tensorflow as tf
from tensorflow import keras
import glob
import cv2
from patchify import patchify, unpatchify
from tqdm.notebook import tqdm

np.set_printoptions(threshold=sys.maxsize)

2023-04-14 13:08:54.138158: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.2


In [ ]:
def get_base_classification_path(file_path, base_preds):
    file_name = os.path.basename(file_path)
    file_name_no_ext = os.path.splitext(file_name)[0]
    return os.path.join(base_preds, file_name_no_ext)

def get_color_classification_path(file_path, color_ext):
    file_name = os.path.basename(file_path)
    file_name_no_ext = os.path.splitext(file_name)[0]
    return os.path.join(color_ext, file_name_no_ext)

def classification_files_exist(cnnpred_file, colorpred_file):
    return os.path.exists(cnnpred_file) and os.path.exists(colorpred_file)

def get_fusion_path(file_path, fusion_ext):
    file_name = os.path.basename(file_path)
    file_name_no_ext = os.path.splitext(file_name)[0]
    return os.path.join(fusion_ext, file_name_no_ext)

def process_images(cnnpred, colorpred):
    """Replace connected components in base predictions with connected components from color predictions."""
    # Find connected components in colorpred and cnnpred
    nb_lbl_color, lbl_color = cv2.connectedComponents(colorpred, connectivity=8)
    nb_lbl_cnn, lbl_cnn = cv2.connectedComponents(cnnpred, connectivity=8)

    # Create an empty array for the output
    output = np.zeros_like(cnnpred)

    # Iterate through the connected components in cnnpred
    for i in range(1, nb_lbl_cnn):
        # Get the mask for the current connected component in cnnpred
        cnn_component_mask = (lbl_cnn == i)

        # Find all the connected components in colorpred that intersect with the current connected component in cnnpred
        color_component_labels = np.unique(lbl_color[cnn_component_mask])
        color_component_labels = color_component_labels[color_component_labels != 0]  # Ignore the background component

        # Replace the current connected component in cnnpred with the corresponding connected components from colorpred
        for color_label in color_component_labels:
            output[(lbl_color == color_label)] = 255

    return output

#### Color extraction function for this database

The hue component is used for colour extraction.  
Values are chosen accordingly to the database. 

In [ ]:
def colorextraction(images_list, color_ext):
    def extractRColor(src):
        """Extract the red component in an image and display it"""

        src_hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)
        
        # # Display the HSV image
        # plt.imshow(cv2.cvtColor(src_hsv, cv2.COLOR_HSV2RGB))
        # plt.title('HSV Image')
        # plt.show()

        lower_red1 = np.array([160,40,140])
        upper_red1 = np.array([180,255,255])

        lower_red2 = np.array([0,40,140]) # empirically determined that 140 was the best threshold for OS
        upper_red2 = np.array([12,255,255])

        mask = cv2.add(cv2.inRange(src_hsv, lower_red1, upper_red1), 
                    cv2.inRange(src_hsv, lower_red2, upper_red2))

        # # Display the initial mask
        # plt.imshow(mask, cmap='gray')
        # plt.title('Initial Mask')
        # plt.show()

        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        kernel_2 = np.ones((2,2), np.uint8)
        kernel_hm = np.array((
            [0, -1, 0],
            [-1, 1, -1],
            [0, -1, 0]), dtype="int")

        kernel_hm_diag = np.array((
            [-1, 0, -1],
            [0, 1, 0],
            [-1, 0, -1]), dtype="int")

        res_hm = cv2.morphologyEx(mask, cv2.MORPH_HITMISS, kernel_hm)
        res_hm_diag = cv2.morphologyEx(mask, cv2.MORPH_HITMISS, kernel_hm_diag)
        
        # # Display hit-miss results
        # plt.imshow(res_hm, cmap='gray')
        # plt.title('Hit-miss result (kernel_hm)')
        # plt.show()

        # plt.imshow(res_hm_diag, cmap='gray')
        # plt.title('Hit-miss result (kernel_hm_diag)')
        # plt.show()

        mask = cv2.subtract(mask, res_hm)
        mask = cv2.subtract(mask, res_hm_diag)

        # # Display mask after hit-miss subtraction
        # plt.imshow(mask, cmap='gray')
        # plt.title('Mask after hit-miss subtraction')

        res_hm = cv2.morphologyEx(mask, cv2.MORPH_HITMISS, kernel_hm)
        res_hm_diag = cv2.morphologyEx(mask, cv2.MORPH_HITMISS, kernel_hm_diag)
        
        # # Display second hit-miss results
        # plt.imshow(res_hm, cmap='gray')
        # plt.title('Second hit-miss result (kernel_hm)')
        # plt.show()

        # plt.imshow(res_hm_diag, cmap='gray')
        # plt.title('Second hit-miss result (kernel_hm_diag)')
        # plt.show()

        mask = cv2.subtract(mask, res_hm)
        mask = cv2.subtract(mask, res_hm_diag)

        # # Display mask after second hit-miss subtraction
        # plt.imshow(mask, cmap='gray')
        # plt.title('Mask after second hit-miss subtraction')
        # plt.show()

        lower_white = np.array([253,253,253])
        upper_white = np.array([255,255,255])
        src_bg = cv2.inRange(src, lower_white, upper_white)
        
        # # Display white background mask
        # plt.imshow(src_bg, cmap='gray')
        # plt.title('White background mask')
        # plt.show()

        src_final = 255 - cv2.add(255 - mask, src_bg)

        # # Display the final result
        # plt.imshow(src_final, cmap='gray')
        # plt.title('Final result')
        # plt.show()

        return src_final

    for filename in tqdm(images_list, total=len(images_list), position=0, leave=True):
        filename_wo_path = filename.split('/')[-1].split('.')[0]
        large_image = cv2.imread(filename)
        red = extractRColor(large_image)
        out_path = os.path.join(color_ext, filename_wo_path +'.png')
        cv2.imwrite(out_path, red)

    return 'Colour extraction completed'

# PATCHIFY CODE START HERE

## Read Image File

In [7]:
from tqdm.auto import tqdm

Modify the path to the model accordingly.

In [4]:
#model = tf.keras.models.load_model('/home/alexis/workspace/notebook/models/model_1.h5')
model = tf.keras.models.load_model('/home/alexis/workspace/notebook/models/model_1_cus.h5')

2023-04-14 13:08:58.517325: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2023-04-14 13:08:58.584331: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:0b:00.0 name: Quadro RTX 5000 computeCapability: 7.5
coreClock: 1.815GHz coreCount: 48 deviceMemorySize: 15.74GiB deviceMemoryBandwidth: 417.29GiB/s
2023-04-14 13:08:58.584432: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.2
2023-04-14 13:08:58.589515: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2023-04-14 13:08:58.594200: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2023-04-14 13:08:58.595415: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcurand.so.10


In [5]:
# MODIFY the path to the image database accordingly
base = r'/home/alexis/workspace/DATA/OS/MISSING_FILES'
base_classif = r'/home/alexis/workspace/DATA/OS/MISSING_FILES/PREDS'
images_list = glob.glob(os.path.join(base, '*.jpg'))
images_list.sort()
#images_list = images_list[76:]
print(len(images_list))

2


In [8]:
for filename in tqdm(images_list, total=len(images_list), position=0, leave=False):
    filename_wo_path = filename.split('/')[-1].split('.')[0]
    print(filename)
    Large_image = cv2.imread(filename, 1)
    Large_image = cv2.cvtColor(Large_image, cv2.COLOR_RGB2BGR) 
    Large_Image=np.array(Large_image)
    
    img_shape = Large_Image.shape

    ## Create Container to hold Image for proper step size
    remainderW =  (Large_Image.shape[0] - 256) % 256
    remainderH =  (Large_Image.shape[1] - 256) % 256

    if remainderW != 0:
        width= Large_Image.shape[0] -remainderW +256
    else:
        width = Large_Image.shape[0]

    if remainderH != 0:
        height= Large_Image.shape[1] -remainderH +256
    else:
        height = Large_Image.shape[1]

    
    container = np.zeros((width, height,3), dtype=int)
    cont_shape = container.shape
    container[0:Large_Image.shape[0], 0:Large_Image.shape[1],:] = Large_Image[0:Large_Image.shape[0],0:Large_Image.shape[1],:]
    #plt.imshow(container)

    del Large_Image
    
    ## Create Patches
    patches= patchify(container, (256 ,256,3),step=256)
    patches.shape
    patches1=patches.reshape(patches.shape[0]*patches.shape[1]*patches.shape[2], 256,256, 3)
    patches1.shape

    ## Perform Prediction on Patches
    #test_pred = model.predict(patches1, verbose=1)

    #if memory problems separate the base
    maxSize_batch = 1000
    n = patches1.shape[0] // maxSize_batch
    m = patches1.shape[0] % maxSize_batch
    
    # if n > 0:
    #     test_pred = model.predict(patches1[:maxSize_batch], verbose=1)

    #     for i in range(1, n):
    #         test_temp = model.predict(patches1[i*maxSize_batch:(i+1)*maxSize_batch], verbose=1)
    #         test_pred = np.concatenate((test_pred, test_temp))

    #     test_temp = model.predict(patches1[n*maxSize_batch:], verbose=1)
    #     test_pred = np.concatenate((test_pred, test_temp))
    # else:
    #     test_pred = model.predict(patches1[:m], verbose=1)

    max_batch_size = 1000
    num_batches = (patches1.shape[0] + max_batch_size - 1) // max_batch_size
    test_pred = np.empty((0, 256, 256, 1))

    for i in range(num_batches):
        start_idx = i * max_batch_size
        end_idx = min((i + 1) * max_batch_size, patches1.shape[0])
        batch_patches = patches1[start_idx:end_idx]
        batch_pred = model.predict(batch_patches, verbose=1)
        batch_pred_threshold = batch_pred > 0.7
        batch_pred_threshold = batch_pred_threshold.astype(int)
        batch_pred_threshold = batch_pred_threshold.reshape((batch_patches.shape[0], 256, 256, 1))
        test_pred = np.concatenate((test_pred, batch_pred_threshold))

    test_pred_threshold = test_pred > 0.7

    ## Consensus
    test_pred_threshold = test_pred_threshold.astype(int)
    test_pred_threshold = test_pred_threshold.reshape((patches1.shape[0], 256, 256, 1))

    del container
    
    ## Merge Pacthes
    #test_pred_threshold=final_pred
    #print(test_pred_threshold.shape , patches1.shape)
    merge_patches= patches1.reshape(patches.shape[0],patches.shape[1],patches.shape[2], 256,256, 3)
    merge_mask = test_pred_threshold.reshape(patches.shape[0],patches.shape[1],patches.shape[2], 256,256, 1)
    merge_patches=unpatchify(merge_patches,cont_shape)
    merge_masks=unpatchify(merge_mask,(cont_shape[0],cont_shape[1],1))
    merge_masks = merge_masks.astype('uint8')
    merge_masks[merge_masks==1]=255
    merge_masks = merge_masks[0:img_shape[0], 0:img_shape[1],:]

    merge_masks[merge_masks>40]=255
    merge_masks[merge_masks<40]=0
    
    del merge_patches
    del merge_mask

    im_path = os.path.join(base_classif, filename_wo_path +'.tif')
    cv2.imwrite(im_path,merge_masks)

  0%|          | 0/2 [00:00<?, ?it/s]

/home/alexis/workspace/DATA/OS/MISSING_FILES/Ordnance_Survey_Drawings_-_Christchurch_Bay_(OSD_75-1).jpg


2023-04-14 13:09:45.125082: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7
2023-04-14 13:09:50.924111: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10


7/7 [==============================] - 2s 333ms/step


 50%|█████     | 1/2 [00:23<00:23, 23.66s/it]

/home/alexis/workspace/DATA/OS/MISSING_FILES/Ordnance_Survey_Drawings_-_Longnor_(OSD_350).jpg
8/8 [==============================] - 4s 494ms/step


## Color extraction

In [10]:
# MODIFY the path to the image database accordingly
base = r'/home/alexis/workspace/DATA/OS/MISSING_FILES/'
colour_ext = r'/home/alexis/workspace/DATA/OS/MISSING_FILES/COLOR_PREDS'

images_list = glob.glob(os.path.join(base, '*.jpg'))
images_list.sort()

colorextraction(images_list, colour_ext)
    

100%|██████████| 2/2 [00:03<00:00,  1.86s/it]


'Colour extraction completed'

# Fusion of color extraction and cnn prediction

In [12]:
from PIL import Image

#Define paths
# MODIFY the path to the image database accordingly
base_preds = r'/home/alexis/workspace/DATA/OS/MISSING_FILES/PREDS'
color_ext = r'/home/alexis/workspace/DATA/OS/MISSING_FILES/COLOR_PREDS'
fusion_ext = r'/home/alexis/workspace/DATA/OS/MISSING_FILES/FUSION'

# Get the list of image files
images_list = glob.glob(os.path.join(base_preds, '*).tif'))
images_list.sort()

new_image_list = []

# Filter image list by checking the existence of classification files
for file_path in images_list:
    colorpred_file = get_color_classification_path(file_path, color_ext) + ".png"
    cnnpred_file = get_base_classification_path(file_path, base_preds) + ".tif"

    if classification_files_exist(cnnpred_file, colorpred_file):
        new_image_list.append(get_base_classification_path(file_path, base_preds))

print(len(new_image_list), "images have predictions and colour extraction")

# Processing
for base_pred_file in tqdm(new_image_list, total=len(new_image_list), position=0, leave=True):
    try:
        # extract filename from base_pred_file wihtout path and extension
        base_fusion_file = get_fusion_path(base_pred_file, fusion_ext)
        base_color_file = get_color_classification_path(base_pred_file, color_ext)
        print("Processing file: " + base_pred_file)

        #if the file has already been processed, skip it
        #if os.path.exists(base_fusion_file + '.tif'):
            #continue

        cnnpred = Image.open(base_pred_file + '.tif')            
        cnnpred = cnnpred.convert('L')
        cnnpred = np.array(cnnpred)
        colorpred = cv2.imread(base_color_file + '.png',0)
        src_final = process_images(cnnpred, colorpred)
        
        # Saving
        base_pred_file = os.path.basename(base_pred_file)
        print("Saving file: " + base_fusion_file + '.tif')
        cv2.imwrite(base_fusion_file + '.tif', src_final)

    except Exception as e:
        print(e)
        print("Error processing file: " + base_pred_file)


3 images have predictions and colour extraction


  0%|          | 0/3 [00:00<?, ?it/s]

Processing file: /home/alexis/workspace/DATA/OS/MISSING_FILES/PREDS/Ordnance_Survey_Drawings_-_Christchurch_Bay_(OSD_75-1)


 33%|███▎      | 1/3 [05:56<11:53, 356.52s/it]

Saving file: /home/alexis/workspace/DATA/OS/MISSING_FILES/FUSION/Ordnance_Survey_Drawings_-_Christchurch_Bay_(OSD_75-1).tif
Processing file: /home/alexis/workspace/DATA/OS/MISSING_FILES/PREDS/Ordnance_Survey_Drawings_-_Longnor_(OSD_350)


 67%|██████▋   | 2/3 [07:35<03:25, 205.25s/it]

Saving file: /home/alexis/workspace/DATA/OS/MISSING_FILES/FUSION/Ordnance_Survey_Drawings_-_Longnor_(OSD_350).tif
Processing file: /home/alexis/workspace/DATA/OS/MISSING_FILES/PREDS/Ordnance_Survey_Drawings_-_Luton_(OSD_148)


100%|██████████| 3/3 [10:35<00:00, 211.87s/it]

Saving file: /home/alexis/workspace/DATA/OS/MISSING_FILES/FUSION/Ordnance_Survey_Drawings_-_Luton_(OSD_148).tif


# Detect lines in image prediction

In [ ]:
import subprocess

# Set environment variable to access opencv static libraries
cv_lib_dir = '../opencv4/build/install_folder/lib:'
if os.getenv('LD_LIBRARY_PATH') == None:
    os.environ['LD_LIBRARY_PATH'] = cv_lib_dir
else:
    os.environ['LD_LIBRARY_PATH'] = cv_lib_dir + os.getenv('LD_LIBRARY_PATH')

print(os.getenv('LD_LIBRARY_PATH'))

In [ ]:
base_classif = 'Image/OS_SURVEY_CLASSIF'
images_list = glob.glob(os.path.join(base_classif, '*_fusion_1_pred.png'))
images_list.sort()
# images_list = images_list[388:]
print(len(images_list))

In [ ]:
base_rep_flash = "yfaula/yfaula_app/build/"
base_results_flash = "yfaula/yfaula_app/images/"

compteur = 0

for filename in tqdm(images_list, total=len(images_list), position=0, leave=True):
    rproc = subprocess.run(["./"+ base_rep_flash + "flash", "-p="+base_rep_flash + "flash_default_parameters.txt", 
                            "--output_dir=" + base_results_flash, filename], capture_output=True)
    if rproc.returncode == 0:
        compteur += 1
    else:
        print( 'Problem with : ', filename)
        print(rproc.stderr)
        
print('NB processed images: ', compteur, '/', len(images_list))

## We use the same image to generate the final results

In [ ]:
base_results_flash = "Image/os_survey_flashlines/"
base_classif = 'Image/OS_SURVEY_CLASSIF/'
images_list = glob.glob(os.path.join(base_classif, '*_fusion_1_pred.png'))
images_list.sort()
#images_list = images_list[388:]
print(len(images_list))

In [ ]:
compteur=0

for filename in tqdm(images_list, total=len(images_list), position=0, leave=True):
    im_flash_path = base_results_flash + filename.split('/')[-1].split('.')[0] + "_flashlines.png"
    
    pred = cv2.imread(filename, 0)
    
    if os.path.exists(im_flash_path):
        lines = cv2.imread(im_flash_path, 0)
        
        ### POST PROCESSING OF LINE DETECTION : 2 possibilities :
        ## change the if condition in order to choose a solution (False or True)
        ## 1) (SOLTUION PROPOSAL 1) enlarge the detection to the adjacent pixels
        if False:
            nb_lbl, lbl = cv2.connectedComponents(pred,connectivity=8)

            #pred_merge_and  = cv2.bitwise_and(pred, cnnpred)

            counter_lbl = [ 0 ] * nb_lbl
            counter_lbl_lines = [ 0 ] * nb_lbl

            for i in range(lbl.shape[0]):
                for j in range(lbl.shape[1]):
                    if pred[i, j]:
                        counter_lbl[lbl[i, j]] += 1
                    if lines[i, j] and pred[i, j]:
                        counter_lbl_lines[lbl[i, j]] += 1

            for i in range(pred.shape[0]):
                for j in range(pred.shape[1]):
                    # between .20 and .33 because of flash mask size
                    if pred[i, j] > 0 and counter_lbl[lbl[i, j]] > 0:
                        pred[i, j] = 0 if counter_lbl_lines[lbl[i, j]]/counter_lbl[lbl[i, j]] > 0.25  else 255

            pred_final = pred
            
        else:
        ## 2) (BASIC SOLUTION) enlarge the detection with morpholgy operations
            ker = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (6, 6)) # depends on the size of flash kernel (7, 7)
            lines = cv2.dilate(lines, ker, iterations=1)
            
            mask_flash2 = cv2.bitwise_and(lines, pred)
            pred_final = cv2.bitwise_xor(pred, mask_flash2)
        
        compteur+=1
    else:
        pred_final = pred

    cv2.imwrite(base_classif + filename.split('/')[-1].split('.')[0] + '_final.tif', pred_final)

print('NB lines elimination: ', compteur)

## (OPTIONAL) A try to remove lines with Hough transform

In [ ]:
import cv2
import numpy as np
import os
import multiprocessing
from queue import Queue
from tqdm.notebook import tqdm

os.chdir('/home/alexis/workspace/BACKUP_HDD/images/OS/OS_SURVEY_CLASSIF')
path_out = r'/home/alexis/workspace/BACKUP_HDD/images/OS/LINE_REMOVED/'

minLineLength = 10
maxLineGap = 100
threshold = 500

# define a function to detect paralell lines and merge them
def merge_lines(img, image_name):

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply a Canny edge detector to the image
    edges = cv2.Canny(gray, 50, 150)

    # Apply morphological closing operation to fill in gaps between lines
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    # Apply the Hough transform to detect lines
    lines = cv2.HoughLinesP(closed, 1, np.pi/180, 100, minLineLength=minLineLength, maxLineGap=maxLineGap)

    # Extract the x, y coordinates of the lines
    coords = []
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            coords.append([(x1, y1), (x2, y2)])

    # Remove invalid lines
    coords = [line for line in coords if line is not None]

    # Merge lines that are both parallel and intersect
    new_coords = []
    for i in range(len(coords)):
        if coords[i] is None:
            continue
        for j in range(i + 1, len(coords)):
            if coords[j] is None:
                continue
            line1 = coords[i]
            line2 = coords[j]
            if line1 is None:
                continue           
            p1, p2 = line1
            p3, p4 = line2
            d1 = np.array(p2) - np.array(p1)
            d2 = np.array(p4) - np.array(p3)
            norm_d1 = np.linalg.norm(d1)
            norm_d2 = np.linalg.norm(d2)
            if norm_d1 > 0 and norm_d2 > 0:
                cos_theta = np.dot(d1, d2) / (norm_d1 * norm_d2)
                if abs(cos_theta) > 0.95:  # threshold for parallel lines
                    d = np.cross(d1, d2)
                    if abs(d) > 1e-10:  # threshold for intersecting lines
                        p1 = np.array(p1).ravel()
                        p2 = np.array(p2).ravel()
                        t1 = np.cross(p3 - p1, d2) / d
                        t2 = np.cross(p1 - p3, d1) / (-d)
                        if 0 <= t1 <= norm_d2 and 0 <= t2 <= norm_d1:
                            intersection = np.array(p1) + t1 * d1 / norm_d1
                            new_coords.append([p1, intersection, p2])
                            new_coords.append([p3, intersection, p4])
                            # Remove processed lines from the list
                            coords[i] = None
                            coords[j] = None

    # Add remaining unprocessed lines to the list
    for line in coords:
        if line is not None:
            new_coords.append(line)

    # Draw the merged lines on the image
    img_lines = img.copy()
    for line in new_coords:
        if len(line) == 2:
            x1, y1 = line[0]
            x2, y2 = line[1]
            # filter out lines that are too short (shorter than threshold)
            if np.linalg.norm(np.array(line[0]) - np.array(line[1])) > threshold:
                cv2.line(img_lines, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 0), 3)
        elif len(line) == 3:
            x1, y1 = line[0]
            x2, y2 = line[2]
            # filter out lines that are too short (shorter than threshold)
            if np.linalg.norm(np.array(line[0]) - np.array(line[2])) > threshold:
                cv2.line(img_lines, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 0), 3)
    return(img_lines, image_name)

# define a function to detect connected groups of pixels 
def dilate_img(img):
    # convert image to grayscale
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # threshold image to create binary mask
    ret, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # find connected components in binary mask
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh, connectivity=8)

    # create new mask with only large enough components
    new_mask = np.zeros(thresh.shape, dtype=np.uint8)
    for i in range(1, num_labels):
        if stats[i, cv2.CC_STAT_AREA] >= 150:
            new_mask[labels == i] = 255

    # dilate the large enough components
    kernel = np.ones((75, 75), np.uint8) 
    dilated_mask = cv2.dilate(new_mask, kernel)

    # apply the new mask to the input image
    dilated_img = cv2.bitwise_and(img, dilated_mask)
    return dilated_img

def save_image(img, image_name):
    image_name = image_name.replace('_fusion_1_pred_final', '')
    image_path_out = os.path.join(path_out, f"{image_name}_CLEANED.png")
    try:
        cv2.imwrite(image_path_out, img)
    except Exception as e:
        print(f"An error occurred while saving the image: {e}")

# define a function to detect lines using LSD
def lsd_line_detector(img):

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 
    # Apply LSD to detect line segments
    #lsd = cv2.createLineSegmentDetector(0)
    lsd = cv2.ximgproc.createFastLineDetector()
    lines = lsd.detect(gray)

    # create a mask with the detected lines of the same size as the image
    mask = np.zeros_like(img, dtype=np.uint8)
    mask = lsd.drawSegments(mask, lines)

    # draw the lines on the image
    img_lines = img.copy()
    img_lines = lsd.drawSegments(img_lines, lines)

    # save the image with the detected lines
    #image_name = image_name.replace('_fusion_1_pred_final', '')
    #image_path_out = os.path.join(path_out, f"{image_name}_LSD.png")
    #cv2.imwrite(image_path_out, img_lines)

    # prolong each lines by 30 pixels in both directions
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = list(map(int, np.ravel(np.asarray(line, dtype=int))))
            d = np.array([x2 - x1, y2 - y1])
            d = d / np.linalg.norm(d)   
            x1 = int(x1 - d[0])
            y1 = int(y1 - d[1])
            x2 = int(x2 + d[0])
            y2 = int(y2 + d[1])
            cv2.line(mask, (x1, y1), (x2, y2), (255, 255, 255), 3)

    #combine as one element the lines that are close to each other
    mask = cv2.dilate(mask, np.ones((5, 5), np.uint8), iterations=5) 

    gray_mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    contours, hierarchy = cv2.findContours(gray_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    filtered_contours = []
    heights = []
    widths = []
    dilated_mask = dilate_img(img)

    for contour in contours:
        # Create a binary mask for the current contour
        contour_mask = np.zeros_like(gray_mask)
        cv2.drawContours(contour_mask, [contour], -1, 255, -1)

        # Apply bitwise AND operation to get the pixels inside the contour
        contour_pixels = cv2.bitwise_and(dilated_mask, contour_mask)

        # Calculate the number of white pixels in the contour
        white_pixels = cv2.countNonZero(contour_pixels)

        #Calculate the total number of pixels in the contour
        total_pixels = cv2.contourArea(contour)

        # Calculate the ratio between the number of white pixels and the area of the contour
        ratio = round((white_pixels / total_pixels*100), 2)
        
        # Calculate the bounding box of the contour
        rect = cv2.minAreaRect(contour)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        w = int(rect[1][0])
        h = int(rect[1][1])
        heights.append(h)
        widths.append(w)
        aspect_ratio = round(float(w) / h, 2)

        # select contour with lower level of white pixels density:
        if w > 200 or h > 200: 
            if ratio < 14:
                #if needed print the ratio on the image next to the shape
                #cv2.putText(img, str(ratio), (x+50, y+50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
                filtered_contours.append(contour)
        
        elif w > 100 or h > 100:
            # then filter out contours that are not thin or wide enough
            if aspect_ratio < 0.3 or aspect_ratio > 3:
                #cv2.putText(img, str(aspect_ratio), (x+50, y+50), cv2.FONT_HERSHEY_SIMPLEX, 2, (225, 255, 0), 2)
                filtered_contours.append(contour)
        else:
            if w > 250 or h > 250:
            # then filter out contours that are not thin or wide enough
                aspect_ratio = round(float(w) / h, 2)
                if aspect_ratio < 0.8 or aspect_ratio > 1.2:
                    #cv2.putText(img, str(aspect_ratio), (x+50, y+50), cv2.FONT_HERSHEY_SIMPLEX, 2, (225, 255, 0), 2)
                    filtered_contours.append(contour)
                            
    # fill the inside of the contours with black pixels
    img_filtered = cv2.drawContours(img, filtered_contours, -1, (0, 0, 0), -1)

    return img_filtered

# LSD line detector #2
def lsd_line_detector2(img, img_to_apply, dist_thresh):

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 
    # Apply LSD to detect line segments
    #lsd = cv2.createLineSegmentDetector(0)
    lsd = cv2.ximgproc.createFastLineDetector()
    lines = lsd.detect(gray)

    # create a mask with the detected lines of the same size as the image
    mask = np.zeros_like(img, dtype=np.uint8)
    mask = lsd.drawSegments(mask, lines)

    # draw the lines on the image
    img_lines = img.copy()
    img_lines = lsd.drawSegments(img_lines, lines)

    # prolong each lines by some pixels in both directions
    if lines is not None: 
        for line in lines:
            x1, y1, x2, y2 = list(map(int, np.ravel(np.asarray(line, dtype=int))))
            d = np.array([x2 - x1, y2 - y1])
            d = d / np.linalg.norm(d)
            d = d * dist_thresh
            x1 = int(x1 - d[0])
            y1 = int(y1 - d[1])
            x2 = int(x2 + d[0])
            y2 = int(y2 + d[1])
            cv2.line(mask, (x1, y1), (x2, y2), (255, 255, 255), 3)

    #combine as one element the lines that are close to each other
    mask = cv2.dilate(mask, np.ones((3, 3), np.uint8), iterations=2) 
    gray_mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    contours, hierarchy = cv2.findContours(gray_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    filtered_contours = []
    heights = []
    widths = []

    for contour in contours:
        # Create a binary mask for the current contour
        contour_mask = np.zeros_like(gray_mask)
        cv2.drawContours(contour_mask, [contour], -1, 255, -1)

        # Calculate the bounding box of the contour
        rect = cv2.minAreaRect(contour)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        w = int(rect[1][0])
        h = int(rect[1][1])
        heights.append(h)
        widths.append(w)
        aspect_ratio = round(float(w) / h, 2)

        # Filter out contours that are not thin or wide enough
        if w > 100 or h > 100:
            if aspect_ratio < 0.2 or aspect_ratio > 5:
                filtered_contours.append(contour)
                            
    # fill the inside of the contours with black pixels and draw the contours on the image
    img_filtered = cv2.drawContours(img_to_apply, filtered_contours, -1, (0, 0, 0), -1)

    return img_filtered

def process_image(image_name, queue):
    img = cv2.imread(image_name)

    # merge parallel lines
    img_merged, _ = merge_lines(img, image_name)

    # apply LSD line detector
    lsd_filtered = lsd_line_detector(img_merged)

    # apply LSD line detector #2
    lsd_filtered2 = lsd_line_detector2(img, lsd_filtered, 30)

    # apply LSD line detector #3
    lsd_filtered3 = lsd_line_detector2(lsd_filtered2, lsd_filtered2, 15)

    # save image
    save_image(lsd_filtered3, image_name)

    # add cleaned image to queue
    queue.put((lsd_filtered3, image_name))

if __name__ == '__main__':
    input_dir = r'/home/alexis/workspace/BACKUP_HDD/images/OS/OS_SURVEY_CLASSIF/'

    # get list of image files
    image_files = [f for f in os.listdir(input_dir) if f.endswith('_fusion_1_pred_final2.tif')]
    #image_files = image_files[:5]

    # create queue to hold processed images
    queue = Queue()

    # create list of processes
    processes = []
    for image_name in image_files:
        process = multiprocessing.Process(target=process_image, args=(image_name, queue))
        processes.append(process)

    # start processes
    for process in processes:
        process.start()

    # wait for processes to finish
    for process in tqdm(processes):
        process.join()

    # retrieve processed images from queue and save to output directory
    while not queue.empty():
        img, image_name = queue.get()


## CALCULATE ACCURACY

In [ ]:
# MODIFY the path to the image database accordingly
base = r'/home/alexis/workspace/DATA/OS/GT/images/'
base_classif = r'/home/alexis/workspace/DATA/OS/GT/preds/'
images_list = glob.glob(os.path.join(base, '*.tif'))
images_list.sort()
print(len(images_list))

In [ ]:
#predict images from OS GT dataset
for filename in tqdm(images_list, total=len(images_list), position=0, leave=False):
    filename_wo_path = filename.split('/')[-1].split('.')[0]
    print(filename)
    Large_image = cv2.imread(filename, 1)
    Large_image = cv2.cvtColor(Large_image, cv2.COLOR_RGB2BGR) 
    Large_Image=np.array(Large_image)
    
    img_shape = Large_Image.shape

    ## Create Container to hold Image for proper step size
    remainderW =  (Large_Image.shape[0] - 256) % 256
    remainderH =  (Large_Image.shape[1] - 256) % 256

    if remainderW != 0:
        width= Large_Image.shape[0] -remainderW +256
    else:
        width = Large_Image.shape[0]

    if remainderH != 0:
        height= Large_Image.shape[1] -remainderH +256
    else:
        height = Large_Image.shape[1]

    
    container = np.zeros((width, height,3), dtype=int)
    cont_shape = container.shape
    container[0:Large_Image.shape[0], 0:Large_Image.shape[1],:] = Large_Image[0:Large_Image.shape[0],0:Large_Image.shape[1],:]
    #plt.imshow(container)

    del Large_Image
    
    ## Create Patches
    patches= patchify(container, (256 ,256,3),step=256)
    patches.shape
    patches1=patches.reshape(patches.shape[0]*patches.shape[1]*patches.shape[2], 256,256, 3)
    patches1.shape

    ## Perform Prediction on Patches
    #test_pred = model.predict(patches1, verbose=1)

    #si probleme mémoire séparer la base
    maxSize_batch = 1000
    n = patches1.shape[0] // maxSize_batch
    m = patches1.shape[0] % maxSize_batch
    
    max_batch_size = 1000
    num_batches = (patches1.shape[0] + max_batch_size - 1) // max_batch_size
    test_pred = np.empty((0, 256, 256, 1))

    for i in range(num_batches):
        start_idx = i * max_batch_size
        end_idx = min((i + 1) * max_batch_size, patches1.shape[0])
        batch_patches = patches1[start_idx:end_idx]
        batch_pred = model.predict(batch_patches, verbose=1)
        batch_pred_threshold = batch_pred > 0.7
        batch_pred_threshold = batch_pred_threshold.astype(int)
        batch_pred_threshold = batch_pred_threshold.reshape((batch_patches.shape[0], 256, 256, 1))
        test_pred = np.concatenate((test_pred, batch_pred_threshold))


    test_pred_threshold = test_pred > 0.7

    ## Consensus
    test_pred_threshold = test_pred_threshold.astype(int)
    test_pred_threshold = test_pred_threshold.reshape((patches1.shape[0], 256, 256, 1))

    del container
    
    ## Merge Pacthes
    #test_pred_threshold=final_pred
    #print(test_pred_threshold.shape , patches1.shape)
    merge_patches= patches1.reshape(patches.shape[0],patches.shape[1],patches.shape[2], 256,256, 3)
    merge_mask = test_pred_threshold.reshape(patches.shape[0],patches.shape[1],patches.shape[2], 256,256, 1)
    merge_patches=unpatchify(merge_patches,cont_shape)
    merge_masks=unpatchify(merge_mask,(cont_shape[0],cont_shape[1],1))
    merge_masks = merge_masks.astype('uint8')
    merge_masks[merge_masks==1]=255
    merge_masks = merge_masks[0:img_shape[0], 0:img_shape[1],:]

    merge_masks[merge_masks>40]=255
    merge_masks[merge_masks<40]=0
    
    del merge_patches
    del merge_mask


    im_path = os.path.join(base_classif, filename_wo_path + '.png')
    cv2.imwrite(im_path,merge_masks)

In [ ]:
def split_image_into_patches(image, patch_size):
    patches = []
    h, w = image.shape
    for i in range(0, h, patch_size):
        for j in range(0, w, patch_size):
            patch = image[i:i+patch_size, j:j+patch_size]
            patches.append(patch)
    return patches

def merge_patches_into_image(patches, original_shape):
    h, w = original_shape
    patch_size = patches[0].shape[0]
    rows = (h + patch_size - 1) // patch_size
    cols = (w + patch_size - 1) // patch_size
    merged_image = np.zeros(original_shape, dtype=np.uint8)

    idx = 0
    for i in range(0, rows):
        for j in range(0, cols):
            patch = patches[idx]
            y1, y2 = i * patch_size, (i + 1) * patch_size
            x1, x2 = j * patch_size, (j + 1) * patch_size
            y2 = min(y2, h)
            x2 = min(x2, w)
            merged_image[y1:y2, x1:x2] = patch[:y2 - y1, :x2 - x1]
            idx += 1

    return merged_image


In [ ]:
from PIL import Image
import cv2
import numpy as np
from patchify import patchify, unpatchify
from tqdm import tqdm
import os

# Define paths
base_preds = r'/home/alexis/workspace/DATA/OS/GT/preds/'
color_ext = r'/home/alexis/workspace/DATA/OS/GT/color/'
fusion_ext = r'/home/alexis/workspace/DATA/OS/GT/fusion/'

colorextraction(images_list, color_ext)

new_image_list = []

# Filter image list by checking the existence of classification files
for file_path in images_list:
    colorpred_file = get_color_classification_path(file_path, color_ext) + ".png"
    cnnpred_file = get_base_classification_path(file_path, base_preds) + ".png"
    print(cnnpred_file, colorpred_file)

    if classification_files_exist(cnnpred_file, colorpred_file):
        new_image_list.append(get_base_classification_path(file_path, base_preds))

print(len(new_image_list), "images have predictions and colour extraction")

# Processing
patch_size = 256

for base_pred_file in tqdm(new_image_list, total=len(new_image_list), position=0, leave=True):
    try:
        # extract filename from base_pred_file without path and extension
        base_fusion_file = get_fusion_path(base_pred_file, fusion_ext)
        base_color_file = get_color_classification_path(base_pred_file, color_ext)
        cnnpred = Image.open(base_pred_file + '.png')
        # convert to 8-bit format
        cnnpred = cnnpred.convert('L')
        cnnpred = np.array(cnnpred)
        colorpred = cv2.imread(base_color_file + '.tif', 0)

        # Split images into smaller patches
        cnnpred_patches = split_image_into_patches(cnnpred, patch_size)
        colorpred_patches = split_image_into_patches(colorpred, patch_size)

        # Apply process_images function on each patch
        processed_patches = []
        for i in tqdm(range(len(cnnpred_patches))):
            processed_patch = process_images(cnnpred_patches[i], colorpred_patches[i])
            processed_patches.append(processed_patch)

        # Merge processed patches
        src_final = merge_patches_into_image(processed_patches, cnnpred.shape)

        # Saving
        base_pred_file = os.path.basename(base_pred_file)
        cv2.imwrite(base_fusion_file + '.tif', src_final)

    except Exception as e:
        print(e)
        print("Error processing file: " + base_pred_file)


In [ ]:
# if the fusion files are flipped use
#flip horzontally and rotate 180 degrees the fusion images
for file in tqdm(os.listdir(fusion_ext), total=len(os.listdir(fusion_ext)), position=0, leave=True):
    try:
        img = cv2.imread(os.path.join(fusion_ext, file), 0)
        img = cv2.flip(img, 1)
        img = cv2.rotate(img, cv2.ROTATE_180)
        cv2.imwrite(os.path.join(fusion_ext, file), img)
    except Exception as e:
        print(e)
        print("Error processing file: " + file)



In [ ]:
from sklearn.metrics import jaccard_score, precision_score, recall_score
from sklearn.metrics import precision_recall_fscore_support

# load the predicted masks
preds = '/home/alexis/workspace/DATA/OS/GT/fusion/'
preds_list = []
for img in os.listdir(preds):
    if img.endswith(".tif"):
        preds_list.append(img)

# load the ground truth masks
gt_folder = '/home/alexis/workspace/DATA/OS/GT/labels/'
gt_list = []
for img in os.listdir(gt_folder):
    if img.endswith(".tif"):
        gt_list.append(img)

# only keep the images that are in both folders
preds_list = [x for x in preds_list if x in gt_list]
gt_list = [x for x in gt_list if x in preds_list]

print('Number of predicted masks: ', len(preds_list))
print('Number of ground truth masks: ', len(gt_list))

# create dictionaries to store the precision, recall, F1 score and IoU, with image names as keys
precision_dict = {}
recall_dict = {}
f1_score_dict = {}
iou_dict = {}

for i in range(len(preds_list)):
    pred = Image.open(os.path.join(preds,preds_list[i]))
    gt = Image.open(os.path.join(gt_folder,gt_list[i]))
    
    # convert to 8-bit format
    pred = pred.convert('L')
    gt = gt.convert('L')
    
    # convert to numpy arrays
    pred = np.array(pred)
    gt = np.array(gt)
    
    # in the gt mask all values above 0 are considered as 1
    gt[gt > 0] = 1
    pred[pred > 0] = 1

    # compute precision, recall, and F1 score, for 0 and 1 values
    precision, recall, f1_score, _ = precision_recall_fscore_support(gt.ravel(), pred.ravel())
    precision_dict[preds_list[i]] = precision
    recall_dict[preds_list[i]] = recall
    f1_score_dict[preds_list[i]] = f1_score

    # compute IoU
    IoU = jaccard_score(gt.ravel(), pred.ravel(), average=None)
    iou_dict[preds_list[i]] = IoU

# print the mean precision, recall, IoU, and F1 score
print('Mean precision: ', np.mean(list(precision_dict.values())))
print('Mean recall: ', np.mean(list(recall_dict.values())))
print('Mean IoU: ', np.mean(list(iou_dict.values())))
print('Mean F1 score: ', np.mean(list(f1_score_dict.values())))